# Build the logbook for SCAN data on new Spectractor

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : May 12th 2022

- For Ronchi

Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2022_03',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/docopt-0.6.2-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Spectractor-2.1-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/rail-0.1.dev0-

In [3]:
DATE = '20220317'
version="v5.0"  # new orientation

In [4]:
# reconstructed / good /xpos /ypos

if version == "v5.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/tmp/ipykernel_889/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
FLAG_HOLO=False
home=os.getenv("HOME")

In [14]:
if FLAG_HOLO:
    image_path="holo"
    logbook_csv=os.path.join(dir_logbooks,"auxtelholologbook_" + DATE +"_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelholologbook_" + DATE +"_" + version + ".xlsx")
else:
    image_path="ronchi170lpmm"
    logbook_csv=os.path.join(dir_logbooks,"auxtelronchilogbook_"+ DATE +"_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelronchilogbook_"+ DATE +"_" + version + ".xlsx")

# Path

In [43]:
HOSTCC=True
disperserfiltername = "empty~ronchi170lpmm"

In [16]:
# Set path depending on which computer running (according HOSTCC)
# /sps/lsst/groups/auxtel/data/2022/empty~ronchi170lpmm/20220316/   


if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
   
    path_images=os.path.join(path_auxtel,"data/2022/empty~ronchi170lpmm/"+DATE)
    path_output_spectractor=os.path.join(path_auxtel,"data/2022/OutputSpectractor/empty~ronchi170lpm"+DATE)
    
else:
    #path_auxtel="/Users/dagoret/DATA/AuxTelData2021"
    path_auxtel="/Users/sylvie/DATA/AuxTelDATA2022/2022"
    #path_spectractor=os.path.join(path_auxtel,"/users/dagoret/MacOSX/github/LSST/Spectractor")
    path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    #path_images=os.path.join(path_auxtel,"holo/quickLookExp_v2/"+DATE)
    path_images=os.path.join(path_auxtel,"ronchi/"+DATE)
    path_output_spectractor=os.path.join(path_auxtel,"OutputSpectractor/ronchi/"+DATE)

In [17]:
dir_images=path_images

In [18]:
all_files=os.listdir(dir_images)

In [19]:
all_files

['exposure_2022031700434_postisrccd.fits',
 'exposure_2022031700295_postisrccd.fits',
 'exposure_2022031700396_postisrccd.fits',
 'exposure_2022031700438_postisrccd.fits',
 'exposure_2022031700437_postisrccd.fits',
 'exposure_2022031700297_postisrccd.fits',
 'exposure_2022031700283_postisrccd.fits',
 'exposure_2022031700333_postisrccd.fits',
 'exposure_2022031700382_postisrccd.fits',
 'exposure_2022031700339_postisrccd.fits',
 'exposure_2022031700389_postisrccd.fits',
 'exposure_2022031700282_postisrccd.fits',
 'exposure_2022031700260_postisrccd.fits',
 'exposure_2022031700312_postisrccd.fits',
 'exposure_2022031700256_postisrccd.fits',
 'exposure_2022031700329_postisrccd.fits',
 'exposure_2022031700288_postisrccd.fits',
 'exposure_2022031700246_postisrccd.fits',
 'exposure_2022031700284_postisrccd.fits',
 'exposure_2022031700235_postisrccd.fits',
 'exposure_2022031700285_postisrccd.fits',
 'exposure_2022031700409_postisrccd.fits',
 'exposure_2022031700432_postisrccd.fits',
 'exposure_

In [20]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [21]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [22]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_postisrccd[.]fits$',filename)
    
    return m[0]

In [23]:
file_tag_forsorting('exposure_2022031700310_postisrccd.fits')

'2022031700310'

In [24]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2022031700434', '2022031700295', '2022031700396', '2022031700438',
       '2022031700437', '2022031700297', '2022031700283', '2022031700333',
       '2022031700382', '2022031700339', '2022031700389', '2022031700282',
       '2022031700260', '2022031700312', '2022031700256', '2022031700329',
       '2022031700288', '2022031700246', '2022031700284', '2022031700235',
       '2022031700285', '2022031700409', '2022031700432', '2022031700390',
       '2022031700416', '2022031700239', '2022031700328', '2022031700370',
       '2022031700426', '2022031700271', '2022031700291', '2022031700349',
       '2022031700402', '2022031700268', '2022031700334', '2022031700431',
       '2022031700247', '2022031700424', '2022031700326', '2022031700353',
       '2022031700234', '2022031700388', '2022031700315', '2022031700313',
       '2022031700270', '2022031700255', '2022031700375', '2022031700354',
       '2022031700362', '2022031700441', '2022031700307', '2022031700267',
       '2022031700243', '

In [25]:
indexes=np.argsort(all_tags)

In [26]:
sorted_files=all_selected_files[indexes]

In [27]:
sorted_files

array(['exposure_2022031700232_postisrccd.fits',
       'exposure_2022031700233_postisrccd.fits',
       'exposure_2022031700234_postisrccd.fits',
       'exposure_2022031700235_postisrccd.fits',
       'exposure_2022031700236_postisrccd.fits',
       'exposure_2022031700239_postisrccd.fits',
       'exposure_2022031700240_postisrccd.fits',
       'exposure_2022031700241_postisrccd.fits',
       'exposure_2022031700242_postisrccd.fits',
       'exposure_2022031700243_postisrccd.fits',
       'exposure_2022031700246_postisrccd.fits',
       'exposure_2022031700247_postisrccd.fits',
       'exposure_2022031700248_postisrccd.fits',
       'exposure_2022031700249_postisrccd.fits',
       'exposure_2022031700250_postisrccd.fits',
       'exposure_2022031700253_postisrccd.fits',
       'exposure_2022031700254_postisrccd.fits',
       'exposure_2022031700255_postisrccd.fits',
       'exposure_2022031700256_postisrccd.fits',
       'exposure_2022031700257_postisrccd.fits',
       'exposure_202

In [28]:
N=len(sorted_files)

# Build the logbook from the headers

In [29]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [30]:
df=pd.DataFrame()

In [31]:
len(all_airmass)

149

In [32]:
N=len(all_indexes)

## Fill with container info

In [33]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [34]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [35]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [36]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2022-03-18T00:48:45.560,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700232_postisrccd.fits,232,mu. col,empty_1,ronchi170lpmm,1.109388,2.0,...,8.9,49.0,8.8,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2022-03-18T00:49:14.065,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700233_postisrccd.fits,233,mu. col,empty_1,ronchi170lpmm,1.110401,2.0,...,8.9,49.0,8.8,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2022-03-18T00:49:35.458,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700234_postisrccd.fits,234,mu. col,empty_1,ronchi170lpmm,1.111106,2.0,...,8.9,49.0,8.8,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2022-03-18T00:49:50.459,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700235_postisrccd.fits,235,mu. col,empty_1,ronchi170lpmm,1.111604,20.0,...,8.9,48.0,9.3,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2022-03-18T00:50:23.360,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700236_postisrccd.fits,236,mu. col,empty_1,ronchi170lpmm,1.112702,20.0,...,8.9,48.0,9.3,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2022-03-18T00:52:14.362,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700239_postisrccd.fits,239,mu. col,empty_1,ronchi170lpmm,1.116356,2.0,...,8.9,49.0,9.2,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2022-03-18T00:52:41.812,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700240_postisrccd.fits,240,mu. col,empty_1,ronchi170lpmm,1.117381,2.0,...,8.9,49.0,9.2,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2022-03-18T00:53:03.613,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700241_postisrccd.fits,241,mu. col,empty_1,ronchi170lpmm,1.118129,2.0,...,8.9,49.0,7.6,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2022-03-18T00:53:18.525,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700242_postisrccd.fits,242,mu. col,empty_1,ronchi170lpmm,1.118637,20.0,...,8.9,49.0,7.6,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2022-03-18T00:53:51.531,/sps/lsst/groups/auxtel/data/2022/empty~ronchi...,exposure_2022031700243_postisrccd.fits,243,mu. col,empty_1,ronchi170lpmm,1.119778,20.0,...,8.9,49.0,8.7,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [39]:
df.to_excel(logbook_xlsx)
df.to_csv(logbook_csv)

In [40]:
logbook_csv

'logbooks/auxtelronchilogbook_20220317_v5.0.csv'